In [1]:
from tensorflow import __version__ as tf_version
from keras import __version__ as keras_version

print(f'keras_version=={keras_version}, tf_version=={tf_version}')

Using TensorFlow backend.


keras_version==2.2.0, tf_version==1.10.0


In [ ]:
from PIL import Image

def png2jpgBlackBG():
    im = Image.open(r"src.png")
    bg = Image.new("RGB", im.size, (255,255,255))
    bg.paste(im,im)
    bg.save(r"dst.jpg")